In [ ]:
import pandas as pd
import numpy as np

from collections import Counter

from haversine import haversine # too slow
from sklearn.model_selection import train_test_split

from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans

from catboost import CatBoostRegressor
from catboost import cv
from catboost import Pool

pd.set_option('display.max_columns', None)


## 1. Загрузка данных

In [ ]:
ls ../../data

In [ ]:
train_df = pd.read_csv("../../data/train.csv")

In [ ]:
test_df = pd.read_csv("../../data/test.csv")

In [ ]:
# 45.040235	38.976080 Краснодар
# 54.734853	55.957865 Уфа
# 53.516670	49.416670 Тольятти
# 56.838633	60.605489 Екатеринбург

In [ ]:
train_df.head()

In [ ]:
train_df["main_id_locality"].value_counts()

In [ ]:
train_df.info()

In [ ]:
mean = (train_df["ETA"] - train_df["RTA"]).mean()
mean

In [ ]:
koeff = (train_df["RTA"].sum() + train_df["RTA"].shape[0]) / train_df["ETA"].sum()
koeff

In [ ]:
(train_df["ETA"] - train_df["RTA"]).hist(figsize=(20,10), bins=100)

Ожидаемое время выше реального (измерение в секундах)

In [ ]:
city_dict = { 
    338: "Краснодар",
    22394: "Тольятти",
    22402: "Уфа",
    22406: "Екатеринбург"
}

train_df['city'] = train_df.main_id_locality.apply(lambda x: city_dict[x])

In [ ]:
(train_df["EDA"] - train_df["RDA"]).hist(figsize=(20,10), bins=50)

In [ ]:
import polyline
polyline.decode(train_df.loc[0, 'route'])[:5]

## 3. Анализ маршрутов

In [ ]:
import geopy.distance
from tqdm import tqdm_notebook
from haversine import haversine

In [ ]:
train_df["route"].dropna().shape[0], train_df["route"].shape[0]

In [ ]:
test_df.info()

Маршрут:
1. Кол-во ребер
2. Средняя длина ребер
3. Сумма длин ребер 
4. Сумма удаленностей от центра каждой точки пути
5. Средняя удаленность от центра
6. Сумма углов между ребрами пути
7. Средний угол 
8. Кол-во кластеров в маршруте
9. Принадлежность к кластеру (one-hot)

Начальна и конечная точки
1. Геогрфическое расстояние от начальной до конечной точки
2. Коэффициент перепробега (отношение реальной длины к расстоянию между точками)
3. Удаленность от центра города начала маршрута
4. Удаленнность от центра города конца маршрута

Город:
1. Средняя длина маршрутов
2. Среднее кол-во ребер 
3. Среднее расстояние между точками начало и конец
4. Средний коэффициент перепробега
5. Средняя сумма удаленностей от центра
6. Средний угол
7. Средняя сумма углов между ребрами пути



Отрисовка на карте маршрута

In [ ]:
coords_1 = (52.2296756, 21.0122287)
coords_2 = (52.406374, 16.9251681)

print(geopy.distance.distance(coords_1, coords_2).km)

In [ ]:
train_df.columns

In [ ]:
# Геогрфическое расстояние от начальной до конечной точки
train_df['haversine'] = train_df.apply(lambda row: haversine((row['latitude'], row['longitude']), 
                             (row['del_latitude'], row['del_longitude'])), axis=1)

In [ ]:
# Удаленность от центра города начала маршрута
train_df['start_offset'] = train_df.apply(lambda row: haversine((row['latitude'], row['longitude']), 
                             (row['center_latitude'], row['center_longitude'])), axis=1)

In [ ]:
# Удаленность от центра города конца маршрута
train_df['finish_offset'] = train_df.apply(lambda row: haversine((row['del_latitude'], row['del_longitude']), 
                             (row['center_latitude'], row['center_longitude'])), axis=1)

In [ ]:
# Коэффициент перепробега (отношение реальной длины к расстоянию между точками)
train_df["koeff_overroute"] = train_df.apply(lambda row: row['EDA'] / row['haversine'], axis=1)

In [ ]:
# %%timeit
# haversine(coords_1, coords_2)

In [ ]:
# %%timeit
# get_distance(coords_1, coords_2)

In [ ]:
def get_route_features(row):
    features = {}
    parts = polyline.decode(row["route"])
    
    parts_count = len(parts)
    parts_distance_sum = 0
    
    for i in range(0, len(parts) - 1):
        parts_distance_sum += get_distance(parts[i], parts[i + 1])

    features["parts_count"] = parts_count
    features["parts_distance_sum"] = parts_distance_sum
    features["parts_distance_avg"] = parts_distance_sum / parts_count
    
    return features

In [ ]:
def get_route_features(row):
    features = {}
   
    try:        
        parts = polyline.decode(row["route"])

        parts_count = len(parts)
        parts_distance_sum = 0

        for i in range(0, len(parts) - 1):
            parts_distance_sum += haversine(parts[i], parts[i + 1])

        features["parts_count"] = parts_count
        features["parts_distance_sum"] = parts_distance_sum
        features["parts_distance_avg"] = parts_distance_sum / parts_count
    except:
        features["parts_count"] = 0
        features["parts_distance_sum"] = 0
        features["parts_distance_avg"] = 0

    return features

In [ ]:
train_route_features = []
for idx, row in tqdm_notebook(train_df.iterrows()):
    train_route_features.append(get_route_features(row))

In [ ]:
train_route_features[0]

In [ ]:
train_route_features_df = pd.DataFrame(train_route_features)

In [ ]:
train_route_features_df.shape

In [ ]:
train_route_features_df.head()

In [ ]:
train_df.shape

In [ ]:
train_df.head()

In [ ]:
train_routes_df = pd.concat([train_df, train_route_features_df], axis=1)

In [ ]:
train_routes_df.shape

## 4. Code

In [ ]:
train_df = pd.read_csv("../../data/train.csv")

In [ ]:
valid_df = pd.read_csv("../../data/validation.csv")

In [ ]:
test_df = pd.read_csv("../../data/test.csv")

In [ ]:
def get_new_features(train_df):
    # Геогрфическое расстояние от начальной до конечной точки
    train_df.loc[:, 'haversine'] = train_df.apply(lambda row: haversine((row['latitude'], row['longitude']), 
                             (row['del_latitude'], row['del_longitude'])), axis=1)
    
    # Удаленность от центра города начала маршрута
    train_df.loc[:, 'start_offset'] = train_df.apply(lambda row: haversine((row['latitude'], row['longitude']), 
                             (row['center_latitude'], row['center_longitude'])), axis=1)

    
    # Удаленность от центра города конца маршрута
    train_df.loc[:, 'finish_offset'] = train_df.apply(lambda row: haversine((row['del_latitude'], row['del_longitude']), 
                             (row['center_latitude'], row['center_longitude'])), axis=1)
    
    # Коэффициент перепробега (отношение реальной длины к расстоянию между точками)
    train_df.loc[:, "koeff_overroute"] = train_df.apply(lambda row: row['EDA'] / row['haversine'], axis=1)
    
    
    train_route_features = []
    for idx, row in tqdm_notebook(train_df.iterrows()):
        train_route_features.append(get_route_features(row))
        
    
    train_route_features_df = pd.DataFrame(train_route_features)
    
    train_routes_df = pd.concat([train_df, train_route_features_df], axis=1)
    
    
    feature_names = ["start_offset", "finish_offset", "koeff_overroute", "parts_count", 
                     "parts_distance_sum", "parts_distance_avg"]
    
    new_features_df = train_routes_df[feature_names]
    
    return new_features_df

In [ ]:
train_df_extended = get_new_features(train_df)

In [ ]:
valid_df_extended = get_new_features(valid_df)

In [ ]:
test_df_extended = get_new_features(test_df)

In [ ]:
!ls  ../../data

In [ ]:
train_df_extended.to_csv("../../data/train_extended.csv", index=False)

In [ ]:
valid_df_extended.to_csv("../../data/valid_extended.csv", index=False)

In [ ]:
test_df_extended.to_csv("../../data/test_extended.csv", index=False)